# [Advent of Code 2020 Day 11](https://adventofcode.com/2020/day/11)

Another Conway's Game of Life it seems...?

## Initial setup

In [1]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing
2D array time.

In [2]:
def parse_input(filename: str) -> Any:

    gen = yield_line(filename)

    result = []

    for line in gen:
        result.append(list(line))

    return result

## Part 1 - Simulation
There isn't really anything mathy to do here I don't think... it's just a simulation question I think...

In [3]:
def count_occupied_seats_once_stabilized(board: list[list[str]]) -> int:

    rows: int = len(board)
    cols: int = len(board[0])

    def get_occupied_seats(i: int, j: int) -> int:
        count = 0
        for dx, dy in DIR.SURR:
            x = i + dx
            y = j + dy
            if (0 <= x < rows) and (0 <= y < cols) and board[x][y] == "#":
                count += 1
        return count

    def get_next_char(i: int, j: int) -> str:
        if board[i][j] == "L":
            if get_occupied_seats(i, j) == 0:
                return "#"
        elif board[i][j] == "#":
            if get_occupied_seats(i, j) >= 4:
                return "L"
        return board[i][j]

    def get_next_board() -> bool:
        next_char: dict[tuple[int, int], str] = {}

        for i in range(len(board)):
            for j in range(len(board[i])):
                if (potential_char := get_next_char(i, j)) != board[i][j]:
                    next_char[(i, j)] = potential_char

        change = False

        for i in range(len(board)):
            for j in range(len(board[i])):
                if (advance := next_char.get((i, j))) is not None:
                    board[i][j] = advance
                    change = True

        return change

    while get_next_board():
        continue

    return sum([row.count("#") for row in board])

In [4]:
%%ipytest
def test_count_occupied_seats_once_stabilized():
    assert count_occupied_seats_once_stabilized(parse_input("example1")) == 37

.                                                                                            [100%]
1 passed in 0.01s


## Part 2 - Eh?
This is going to take a lot of computation. I should only have to modify the `get_occupied_seats` function though. Instead of extending one out in all eight directions, we extend continuously out in all directions until we either hit the wall or a seat (`#` or `L`), only incrementing our counter if it's `#` occupied. We also increase the urinal factor from 4 to 5.

In [5]:
def count_occupied_seats_once_stabilized_crazy(board: list[list[str]]) -> int:

    rows: int = len(board)
    cols: int = len(board[0])

    def get_occupied_seats(i: int, j: int) -> int:
        count = 0
        for dx, dy in DIR.SURR:
            x = i
            y = j
            while True:
                x += dx
                y += dy
                if (0 <= x < rows) and (0 <= y < cols):
                    if board[x][y] == "#":
                        count += 1
                        break
                    elif board[x][y] == "L":
                        break
                else:
                    break
        return count

    def get_next_char(i: int, j: int) -> str:
        if board[i][j] == "L":
            if get_occupied_seats(i, j) == 0:
                return "#"
        elif board[i][j] == "#":
            if get_occupied_seats(i, j) >= 5:
                return "L"
        return board[i][j]

    def get_next_board() -> bool:
        next_char: dict[tuple[int, int], str] = {}

        for i in range(len(board)):
            for j in range(len(board[i])):
                if (potential_char := get_next_char(i, j)) != board[i][j]:
                    next_char[(i, j)] = potential_char

        change = False

        for i in range(len(board)):
            for j in range(len(board[i])):
                if (advance := next_char.get((i, j))) is not None:
                    board[i][j] = advance
                    change = True

        return change

    while get_next_board():
        continue

    return sum([row.count("#") for row in board])

## Main Solver

In [6]:
def solve(prob, filename):
    parsed_input = parse_input(filename)
    if prob == 1:
        return count_occupied_seats_once_stabilized(parsed_input)
    elif prob == 2:
        return count_occupied_seats_once_stabilized_crazy(parsed_input)
    else:
        print("Invalid problem code")
        exit()

In [7]:
%%ipytest
def test_solve():
    assert solve(1, "example1") == 37
    assert solve(1, "input") == 2275
    assert solve(2, "example1") == 26
    assert solve(2, "input") == 2121

.                                                                                            [100%]
1 passed in 2.38s
